In [ ]:
import win32com.client
import subprocess
import time
from openpyxl import Workbook
from openpyxl import load_workbook
import re
from usefulFunctions import *

#poo desde video 26

class sapInterfaceJob():
    def __init__(self):
        pass

    def startSAP(self, environment):
        global sapGuiAuto, application, connection, session, session1, paths, login, user, psw
        wb = load_workbook(r"C:\Users\crist\OneDrive - UNIVERSIDAD NACIONAL DE INGENIERIA\Venado\Cris\Traslado de tesoreria B5\config.xlsx")
        ws = wb['config']
        ws1 = wb['sapLogin']

        paths = {'accountPath': ws['B1'].value,
                'SAPPath': ws['B2'].value,
                'migraPath': ws['B3'].value}
        
        # accountPath = str(paths['accountPath'])
        # SAPPath = str(paths['SAPPath'])
        # migraPath = str(paths['migraPath'])

        

        login = {'user': ws1['B1'].value,
                 'psw': ws1['B2'].value}

        # user = str(login['user'])
        # psw = str(login['psw'])
        
       
        #command2 =r"D:\Program Files (x86)\ERPSAP\SAPgui\saplogon.exe"
        proc = subprocess.Popen([paths['SAPPath'], '-new-tab'])
        time.sleep(2)

        sapGuiAuto = win32com.client.GetObject('SAPGUI')
        if not type(sapGuiAuto) == win32com.client.CDispatch:
            pass

        application = sapGuiAuto.GetScriptingEngine
        if not type(application) == win32com.client.CDispatch:
            sapGuiAuto = None
            pass

        #connection = application.OpenConnection("SAP QAS", True)
        connection = application.OpenConnection(environment, True)
        if not type(connection) == win32com.client.CDispatch:
            application = None
            sapGuiAuto = None
            pass

        session = connection.Children(0)
        #session1 = connection.Children(1)
       
        if not type(session) == win32com.client.CDispatch:
            connection = None
            application = None
            sapGuiAuto = None
            pass

        # if not type(session1) == win32com.client.CDispatch:
        #     connection = None
        #     application = None
        #     sapGuiAuto = None
        #     pass

        session.findById("wnd[0]/usr/txtRSYST-BNAME").text = login['user']
        session.findById("wnd[0]/usr/pwdRSYST-BCODE").text = login['psw']
        session.findById("wnd[0]").sendVKey(0)

    def loadBankAccounts(self):

        # session1.findById("wnd[0]/tbar[0]/okcd").text= "f-02"
        # session1.findById("wnd[0]").sendVKey(0)
        z = today()
        x = f"C:\\Users\\crist\\OneDrive - UNIVERSIDAD NACIONAL DE INGENIERIA\\Venado\\Cris\\Traslado de tesoreria B5\\Cuentas recaudadoras\\{z}\\CUENTAS DE CAJA IVSA.xlsx" 
        y = xlsxFormatting(x)
        wb2 = load_workbook(y)
        ws2 = wb2['CAJAS RECAUDADORAS']
        i = 9
        j = 0
        txtPath = r'C:\Users\crist\OneDrive - UNIVERSIDAD NACIONAL DE INGENIERIA\Venado\Cris\Traslado de tesoreria B5'

        while True:
            session.findById("wnd[0]/tbar[0]/okcd").text = "fbl3n"
            session.findById("wnd[0]").sendVKey(0)
            accountNumber1 = ws2[f'C{i}'].value
            accountNumberStr1 = str(accountNumber1).replace(' ', '')
            accountNumber2 = ws2[f'D{i}'].value
            accountNumberStr2 = str(accountNumber2).replace(' ', '')
            bank =  ws2[f'E{i}'].value
            bank = str(bank).strip()
            asignaciones = []
            ndocs = []
            fechas = []
            cts = []
            importes = []
            superLista = []

            k = 11
            if len(accountNumberStr1)==9 and len(accountNumberStr2)==9 and type(accountNumber1)== int and type(accountNumber2)== int:
                session.findById("wnd[0]/usr/ctxtSD_SAKNR-LOW").text = accountNumberStr1
                session.findById("wnd[0]/usr/ctxtSD_BUKRS-LOW").text = "GV01"
                session.findById("wnd[0]/usr/ctxtSD_BUKRS-LOW").setFocus
                session.findById("wnd[0]/tbar[1]/btn[8]").press()
                for k in range(11, 20):
                    a = f'wnd[0]/usr/lbl[9,{k}]'
                    nd = f'wnd[0]/usr/lbl[28,{k}]'
                    f = f'wnd[0]/usr/lbl[53,{k}]'
                    c = f'wnd[0]/usr/lbl[64,{k}]'
                    im = f'wnd[0]/usr/lbl[67,{k}]'
                    asignacion = session.findById(a).text
                    ndoc = session.findById(nd).text
                    fecha = session.findById(f).text
                    ct = session.findById(c).text
                    importe = session.findById(im).text                
                    rec = session.findById('wnd[0]/usr/lbl[37,1]').text
                    rec = str(rec)
                    txtCabDoc = 'TRASLADO A ' + bank
                    print(txtCabDoc)
                    r2 = re.search('RECAUDADORA', rec).span()
                    r2 = r2[1]
                    r2+=1
                    rec = rec[r2:]
                    rec = rec.strip()
                    rec = rec.replace(' ', '.')
                
                    asignacion = str(asignacion).replace(' ', '')
                    ndoc = str(ndoc).replace(' ', '')
                    fecha = str(fecha).replace(' ', '')
                    l = fecha.index('.')
                    fecha = fecha[:l+3]
                    ct = str(ct).replace(' ', '')
                    importe = str(importe).replace(' ', '')
                    
                    asignaciones.append(asignacion)
                    ndocs.append(ndoc)
                    fechas.append(fecha)
                    cts.append(ct)
                    importes.append(importe)

                    texto = 'LP.TRASPASO ' + rec + ' A ' + bank + ' ' + fecha
            superLista.append(asignaciones)
            superLista.append(ndocs)
            superLista.append(fechas)
            superLista.append(cts)
            superLista.append(importes)
            writeLog('', superLista, txtPath)
# PROCESO -------------------------------------------------------------
                
            break   
        a = superLista[4][0]
        a = a.replace(' ', '')
        a = a.replace('-', '')
        b = superLista[4][1]
        b = b.replace(' ', '')
        b = b.replace('-', '')
        print(a)
        print(b)
        if a == b:
            print('iguales')

if __name__=='__main__':
    environment= "QAS - EHP8 on HANA"
    bot5SapInterface = sapInterfaceJob()
    bot5SapInterface.startSAP(environment)
    bot5SapInterface.loadBankAccounts()


In [17]:
l = [1, 1, 1, 1, 2]
n = l.count(1)
print(n)

4


In [ ]:
import win32com.client
import subprocess
import time
from openpyxl import Workbook
from openpyxl import load_workbook
import re
from usefulFunctions import *

i = 3
j = 0

z = today()
x = f"C:\\Users\\crist\\OneDrive - UNIVERSIDAD NACIONAL DE INGENIERIA\\Venado\\Cris\\Traslado de tesoreria B5\\Cuentas recaudadoras\\{z}\\CUENTAS DE CAJA IVSA.xlsx" 
y = xlsxFormatting(x)
wb2 = load_workbook(y)
ws2 = wb2['CAJAS RECAUDADORAS']
xlsxCellsRange = []

while True:
    accountNumber1 = ws2[f'C{i}'].value
    accountNumberStr1 = str(accountNumber1).replace(' ', '')
    accountNumber2 = ws2[f'D{i}'].value
    accountNumberStr2 = str(accountNumber2).replace(' ', '')
    bank =  ws2[f'E{i}'].value
    bank = str(bank).strip()

    if len(accountNumberStr1)==9 and len(accountNumberStr2)==9 and type(accountNumber1)== int and type(accountNumber2)== int:
        xlsxCellsRange.append(i)
        i+=1
    else:
        i+=1
        j+=1
        if j > 3:
            break
        else:
            continue
print(xlsxCellsRange)

In [ ]:
from usefulFunctions import *
y = r"C:\Users\crist\Desktop"
x = 'holi'
writeLog('', x, y)

In [21]:
for f in range(4):
    print(f)

0
1
2
3


In [ ]:
l = ['110315097', '110315098', '110315099', '110315100', '110315101', '110315102', '110315103', '110315104', '110315105', '110315106', '110315107']






In [1]:
l = 'hola'
l = l.replace('k', '')
print(l)

hola
